# 使用卷积神经网络处理 MNIST 数据

导入相关的数据

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


启动交互式会话，即可以在运行时调整运行图。默认情况下，tensorflow 会把运行图生成出来后完全交给后端处理，以减少数据在后端和 Python 之间传输的开销。因此，默认情况下 Python 只是用于定义运行图的节点，而所有计算都会在 Python 外部运行。

In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()

## 初始化必要的参数

In [3]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

## 初始化权重

创建权重初始化函数，后文将多次用到这个函数进行权重矩阵的初始化。一般来说，这些初始化权重都应该是不全为 0 的随机数，以避免 0 梯度。另外，初始化为正数偏置可以避免“死神经元”的情况。

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

In [5]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 卷积核和池化（Pooling）

tensorflow 在卷积核和池化方面很灵活，可以定义包括如何处理边界、步长等。在这个例子中，我们只用一个普通的例子。卷积核是用步长为 1 并以 0 填充，因此输出的矩阵规模将会是和原始输入一致。池化也是简单的 2x2 最大池化（max pooling）方法。

In [6]:
def conv2d(x, W):
    return tf.nn.conv2d(
        x, W,
        strides=[1, 1, 1, 1],
        padding='SAME'
    )

In [7]:
def max_pool_2x2(x):
    return tf.nn.max_pool(
        x,
        ksize=[1, 2, 2, 1],
        strides=[1, 2, 2, 1], padding='SAME'
    )

## 第一个卷积层

我们将创建第一个卷积层，使用卷积然后最大池化。这个卷积操作将会为每个 5x5 的方块计算 32 个特征。它的权重张量的形状是 `[5, 5, 1, 32]` 。前两维是方块大小，然后是输入通道数量，最后是输出的通道（在图像处理中，每种颜色称为一种通道）。

In [8]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

为了应用这个卷积层，我们首先把 x 的形状改为四维的张量，第二、第三维对应图像的宽和高，最后一维对应到颜色通道。

In [9]:
x_image = tf.reshape(x, [-1,28,28,1])

然后我们对 `x_image` 进行卷积的操作。使用权重张量以及偏置，应用 `ReLU` 函数，最后使用最大池。

In [10]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

## 第二个卷积层

为了使用深度网络，我们堆积几个这种类型的卷积层。第二层将会产生 64 个特征。

In [11]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## 紧密连接的层

现在图片大小已经缩减到 7x7 了，我们增加一个 1024 个神经元的全连接层处理整个图片。

In [12]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

为了减少过拟合风险，需要应用一个 dropout 层。

In [13]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## 输出层（Readout）

最后，我们创建一个 softmax 层，就像前一个简单例子那样。

In [14]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

## 训练并评估模型

训练这个模型和 softmax 类似，差别是这里使用了 ADAM 优化器代替最速下降优化器。

In [15]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(
            feed_dict={
                x: batch[0],
                y_: batch[1],
                keep_prob: 1.0
            }
        )
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.06
step 100, training accuracy 0.86
step 200, training accuracy 0.96
step 300, training accuracy 0.86
step 400, training accuracy 1
step 500, training accuracy 0.94
step 600, training accuracy 1
step 700, training accuracy 0.96
step 800, training accuracy 0.9
step 900, training accuracy 1
step 1000, training accuracy 0.96
step 1100, training accuracy 0.94
step 1200, training accuracy 1
step 1300, training accuracy 1
step 1400, training accuracy 0.96
step 1500, training accuracy 0.98
step 1600, training accuracy 1
step 1700, training accuracy 0.98
step 1800, training accuracy 0.96
step 1900, training accuracy 1
step 2000, training accuracy 0.98
step 2100, training accuracy 0.96
step 2200, training accuracy 0.96
step 2300, training accuracy 1
step 2400, training accuracy 1
step 2500, training accuracy 1
step 2600, training accuracy 0.98
step 2700, training accuracy 1
step 2800, training accuracy 0.98
step 2900, training accuracy 0.94
step 3000, training accura

In [16]:
print("test accuracy %g" % accuracy.eval(
        feed_dict={
            x: mnist.test.images,
            y_: mnist.test.labels,
            keep_prob: 1.0
        }
    ))

test accuracy 0.9917


注：对于小型卷积网络，有没有 dropout 层都是差不多的，dropout 层对训练较大的神经网络有比较明显的作用。